In [712]:

# pip3 install cloudscraper beautifulsoup4
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
import time
import re
from deep_translator import GoogleTranslator
from unidecode import unidecode
import json
from datetime import datetime, timedelta
import random
import requests
from langdetect import detect
import numpy as np

In [713]:
def normalize_text(text):
    # Các từ cần loại bỏ
    text = str(text)
    text = re.sub(r"\bTP\.?\s*", "", text, flags=re.IGNORECASE)
    words_to_remove = ["huyện", "thị xã", "thành phố", "quận", "tỉnh"]
    
    # Loại bỏ các từ không cần thiết
    for word in words_to_remove:
        text = text.lower().replace(word, "")
    
    # Loại bỏ dấu tiếng Việt
    text = unidecode(text)
    # Loại bỏ dấu cách và chuyển về chữ thường
    text = text.replace(" ", "").lower()
    return text


In [714]:
def normalize_text_for_dict(text):
    text = re.sub(r"[\n\t\r]+", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [715]:
def convert_to_camel_case(text):
    # Chuyển tất cả chữ về thường
    text = unidecode(text.lower())
    # Tách các từ dựa trên khoảng trắng
    words = text.split()
    # Viết hoa chữ cái đầu của từng từ và nối chúng lại
    camel_case = ''.join(word.capitalize() for word in words)
    return camel_case

In [716]:
# Hàm để xử lý kiểu dữ liệu không serializable
def default_converter(o):
    if isinstance(o, (np.integer, np.floating)):  # Xử lý kiểu số NumPy
        return int(o)
    elif isinstance(o, (np.ndarray,)):           # Xử lý mảng NumPy (nếu có)
        return o.tolist()
    return str(o)  # Xử lý các kiểu dữ liệu khác

In [717]:
def get_url(soup):
    canonical_tag = soup.find('link', rel='canonical')
    # Lấy giá trị của thuộc tính href
    if canonical_tag and 'href' in canonical_tag.attrs:
        canonical_url = canonical_tag['href']
        return canonical_url
    return None

In [718]:
def extract_id_and_convert(url):
    # Regex để lấy ID giữa dấu . và .html
    match = re.search(r"\.([A-Fa-f0-9]+)\.html", url)
    if match:
        hex_id = match.group(1)  # Lấy ID dạng hex
        dec_id = int(hex_id, 16)  # Chuyển từ hex sang dec
        return dec_id
    return None

In [719]:
def get_heading1(soup):
    canonical_tag = soup.find('h1', class_="title")
    # Lấy giá trị của thuộc tính href
    if canonical_tag:
        return canonical_tag.text.strip()
    return None

In [720]:
def get_info_company(soup):
    link = None
    id = None
    name = None
    canonical_tag = soup.find('a', class_="employer job-company-name")
    # Lấy giá trị của thuộc tính href
    if canonical_tag:
        name = canonical_tag.text.strip()
        link = canonical_tag.get('href')
        if link:
            id = extract_id_and_convert(link)
    return id, link, name
    

In [721]:
def get_dia_diem(soup):
    output = []
    canonical_tag = soup.find('div', class_="map")
    # Lấy giá trị của thuộc tính href
    if canonical_tag:
        canonical_tag1 = canonical_tag.find('p')
        if canonical_tag1:
            for temp in canonical_tag1.find_all('a'):
                output.append(temp.text)
    return output

In [722]:
def convert_number(value):
    # try:
        if not value:
            return None
        number = float(value.replace(",", ""))
        # Kiểm tra nếu là số nguyên
        if number.is_integer():
            return int(number)
        else:
            return number  # Trả về float
    # except Exception:
    #     return value  # Trả về giá trị gốc nếu có lỗi

In [723]:
def extract_number_salary(text):
    # Biểu thức chính quy tìm kiếm số, bao gồm số có dấu phẩy phân cách
    match = re.search(r'[\d,]+', text)
    if match:
        # Loại bỏ dấu ',' và trả về số dưới dạng float
        number = match.group(0)
        return convert_number(number)  # Chuyển đổi thành float
    return None

In [724]:
def processing_salary(text):
    if not text:
        return None, None, None
    if re.search(r'\btr\b', text.lower()):
        don_vi_tien = 'VND'
        text = text.lower().replace('VND', '')
        muc_luong = re.sub(r'\btr\b', '', text.lower())
    elif 'USD' in text:
        don_vi_tien = 'USD'
        muc_luong = text.replace('USD', '')
    else:
        don_vi_tien = None
        muc_luong = text
    muc_luong = muc_luong.split('-')
    if len(muc_luong) > 1:
        muc_luong_den = extract_number_salary(muc_luong[1].strip())
        muc_luong_tu = extract_number_salary(muc_luong[0].strip())
    else:
        if 'trên' in muc_luong[0].strip().lower() or 'từ' in muc_luong[0].strip().lower():
            muc_luong_den = 0
            muc_luong_tu = extract_number_salary(muc_luong[0])
        elif 'dưới' in muc_luong[0].strip().lower() or 'tới' in muc_luong[0].strip().lower() or 'đến' in muc_luong[0].strip().lower():
            muc_luong_tu = 0
            muc_luong_den = extract_number_salary(muc_luong[0])
        else:
            muc_luong_tu = 0
            muc_luong_den = 0
    return muc_luong_tu, muc_luong_den, don_vi_tien

In [725]:
def processing_tuoi(text):
    text_save = text
    text = text.split('-')
    if len(text) > 1:
        tuoiden = extract_number_salary(text[1].strip())
        tuoitu = extract_number_salary(text[0].strip())
        khac = None
    else:
        if 'trên' in text[0].strip().lower() or 'từ' in text[0].strip().lower():
            tuoiden = 0
            tuoitu = extract_number_salary(text[0])
            khac = None
        elif 'dưới' in text[0].strip().lower() or 'tới' in text[0].strip().lower() or 'đến' in text[0].strip().lower():
            tuoitu = 0
            tuoiden = extract_number_salary(text[0])
            khac = None
        else:
            tuoitu = None
            tuoiden = None
            khac =  text_save
    return tuoitu, tuoiden, khac

In [726]:
def get_info_post(soup):
    list_nghe = []
    dict = {}
    canonical_tag = soup.find_all('div', class_="detail-box has-background")
    # Lấy giá trị của thuộc tính href
    if canonical_tag:
        for temp in canonical_tag:
            if temp:
           
                for temp1 in temp.find_all('li'):
                    if temp1 and temp1.find('strong'):
                        if temp1.find('strong').text.strip().lower() == 'ngành nghề':
                            for temp2 in temp1.find_all('a'):
                                list_nghe.append(temp2.text.strip())
                        else:
                            dict[convert_to_camel_case(temp1.find('strong').text.strip().replace('\n', ''))] = (normalize_text_for_dict(temp1.find('p').text.strip()))
    dict["NganhNghe"] = list_nghe
    return dict

In [727]:
def get_phuc_loi(soup):
    array = []
    data = soup.find('ul', class_="welfare-list")
    if data:
        for temp in data.find_all('li'):
            if temp:
                array.append(temp.text.strip())
    return array

In [728]:
def get_info_job(soup):
    try:
        dict = {}
        data = soup.find_all('div', class_="detail-row reset-bullet")
        if data:
            for temp in data:
                title = temp.find(class_="detail-title").text
                if title:
                    temp.find(class_="detail-title").decompose()
                    dict[normalize_text_for_dict(convert_to_camel_case(title.strip().replace('\n', '')))] = normalize_text_for_dict(temp.decode_contents().strip())
        return dict
    except Exception as e:
        print(f"Error processing tag: {e}")

In [729]:
def get_thong_tin_khac(soup):
    dict = {}
    dict_doTuoi = {
        'TuoiTu': None,
        'TuoiDen': None,
        'Khac': None,
    }
    dict['DoTuoi'] = dict_doTuoi
    data = soup.find('div', class_="content_fck")
    if data:
        for temp in data.find_all('li'):
            if temp:
                output = temp.text.strip().split(':')
                # print(output)
                if 'DoTuoi' == convert_to_camel_case(output[0].strip().replace('\n', '')):
                    dict_doTuoi = {
                        'TuoiTu': processing_tuoi(normalize_text_for_dict(output[1].strip()))[0],
                        'TuoiDen': processing_tuoi(normalize_text_for_dict(output[1].strip()))[1],
                        'Khac': processing_tuoi(normalize_text_for_dict(output[1].strip()))[2],
                    }
                    dict['DoTuoi'] = dict_doTuoi 
                else:
                    dict[convert_to_camel_case(output[0].strip().replace('\n', ''))] = normalize_text_for_dict(output[1].strip())
    return dict

In [730]:
def get_key_word(soup):
    output = []
    data = soup.find('div', class_="job-tags")
    if data:
        for temp in data.find_all('li'):
            output.append(normalize_text_for_dict(temp.text))
    return output

In [731]:
def get_dia_chi(soup):
    dia_chi = []
    tinh_thanh = []
    data = soup.find_all('div', class_="place-name")
    if data:
        for temp in data:
            for temp1 in temp.find_all('span'):
                dia_chi.append(normalize_text_for_dict(temp1.text))
            if temp.find('div', class_="place"):
                tinh_thanh.extend([temp.find('div', class_="place").text.strip()] * len(temp.find_all('span')))
    return dia_chi, tinh_thanh

In [732]:
def convert_value(value):
    if not value or value == 'None':
        return None
    try:
        # Kiểm tra nếu là số nguyên
        return int(value)
    except ValueError:
        try:
            # Kiểm tra nếu là số thập phân
            return float(value)
        except ValueError:
            # Nếu không phải số, trả về dưới dạng chuỗi
            return str(value)


In [733]:
def IsThoaThuan(text):
    if processing_salary(text) == (0, 0, None):
        return True
    return False

In [734]:
def is_english(text):
    try:
        # Nhận diện ngôn ngữ của văn bản
        return detect(text) == 'en'
    except:
        return False


In [735]:
def load_df():
    return pd.read_excel(r'C:\Users\My_Pc\Desktop\Thư Viện Pháp Luật final\Crawl tuyển dụng\Code\data_id\vietnamwork_data_setup.xlsx', engine='openpyxl')

In [736]:
def split_address(dia_chi_input):
    if ',' in dia_chi_input:
        dia_chi_input_split = dia_chi_input.split(',')
    elif '-' in dia_chi_input:
        dia_chi_input_split = dia_chi_input.split('-')
    elif '–' in dia_chi_input:
        dia_chi_input_split = dia_chi_input.split('–')
    else:
        # Nếu không có dấu phẩy và dấu gạch ngang, trả về chuỗi ban đầu trong danh sách
        dia_chi_input_split = [dia_chi_input]
    
    # Loại bỏ khoảng trắng thừa cho mỗi phần tử
    dia_chi_input_split = [item.strip() for item in dia_chi_input_split]
    
    return dia_chi_input_split

In [737]:
# Tìm kiếm với nhiều điều kiện
def find_id_tinhThanh(name, df):
    # print(name)
    result = df[
        (df['processing'].apply(lambda x: str(x).lower() in normalize_text(name.lower()))) &
        (df['ParentId'] == 0) &
        (df['Level'] == 1)
    ]

    if not result.empty:
        id_value = result.iloc[0]['Id']
        # parent_id_value = result.iloc[0]['ParentId']
        # level_value = result.iloc[0]['Level']
        name_value = result.iloc[0]['UnitName']
        name_value_processing = result.iloc[0]['processing']
        # print(f"Id: {id_value}, ParentId: {parent_id_value}, Level: {level_value}")
        return id_value, name_value, name_value_processing
    return None, None, None


In [738]:
# Tìm kiếm với nhiều điều kiện
def find_id_tinhQuanHuyen(name, ParentId, df):
    if not name:
        return None, None, None
    name = re.sub(r"Quận 2", "Thủ Đức", name, flags=re.IGNORECASE)
    # name = re.sub(r"\bTP\.?\s*", "", name, flags=re.IGNORECASE)
    result = df[
        (df['processing'].apply(lambda x: str(x).lower() in normalize_text(name.lower()))) &
        (df['ParentId'] == ParentId) &
        (df['Level'] == 2)
    ]

    if not result.empty:
        id_value = result.iloc[0]['Id']
        name_value = result.iloc[0]['UnitName']
        name_value_processing = result.iloc[0]['processing']
        # print(f"Id: {id_value}, ParentId: {parent_id_value}, Level: {level_value}")
        return id_value, name_value, name_value_processing
    return None, None, None


In [739]:
def remove_from_last_match(word, word_list):
    # Duyệt từ cuối mảng về đầu để tìm từ trùng khớp cuối cùng
    for i in range(len(word_list) - 1, -1, -1):
        # print(normalize_text(word))
        # print(normalize_text(word_list[i]))
        if str(normalize_text(word)) in normalize_text(word_list[i]):
            # Cắt mảng từ vị trí i trở đi
            return word_list[:i]
    
    # Nếu không có từ trùng khớp, trả về mảng gốc
    return word_list

In [740]:
def process_address_data(locations, dia_chi_inputs, df):
    if not locations or not dia_chi_inputs:
        return None
    output = []
    output_tu_nhap = []
    for location, dia_chi_input in zip(locations, dia_chi_inputs):
        # print(find_id_tinhThanh(location, df))
        tinhThanh = find_id_tinhThanh(location, df)
        con_lai = remove_from_last_match(tinhThanh[-1], split_address(dia_chi_input))
        if len(con_lai) >= 1:
            tinhQuanHuyen = (find_id_tinhQuanHuyen(con_lai[-1], tinhThanh[0], df))
            if tinhQuanHuyen[-1]:
                con_lai = (remove_from_last_match(tinhQuanHuyen[-1], (con_lai)))
                diaChiDict = {
                    'IdTinhThanh': tinhThanh[0],
                    'TinhThanh': tinhThanh[1],
                    'IdQuanHuyen': tinhQuanHuyen[0],
                    'TinhQuanHuyen':tinhQuanHuyen[1],
                }
                data_con_lai = ','.join(con_lai)
                if data_con_lai:
                    diaChiDict['DiaChi'] = data_con_lai
                    diaChiDict['IsEN'] = is_english(dia_chi_input)
                else:
                    diaChiDict['DiaChi'] = None
                    diaChiDict['IsEN'] = is_english(dia_chi_input)
            else:
                diaChiDict = {
                    'IdTinhThanh': tinhThanh[0],
                    'TinhThanh': tinhThanh[1],
                    'IdQuanHuyen': None,
                    'TinhQuanHuyen': None,
                    'DiaChi': ','.join(con_lai),
                    'IsEN': is_english(dia_chi_input),
                }
        else:
                diaChiDict = {
                'IdTinhThanh': tinhThanh[0],
                'TinhThanh': tinhThanh[1],
                'IdQuanHuyen': None,
                'TinhQuanHuyen':None,
                'DiaChi': None,
                'IsEN': is_english(dia_chi_input),
            }
        if tinhThanh[0] == None:
            output_tu_nhap.append(dia_chi_inputs)       
        else: 
            output.append(diaChiDict)
        dict_final = {
            'DiaDiemCuThe': output,
            'DiaDiemTuNhap': output_tu_nhap,
        }
    return dict_final

In [741]:
# def processing_core(soup):
#     soup = soup.find('div', class_="detail-row")
#     # Tìm tất cả các thẻ có class 'detail-title'
#     titles = soup.find_all(class_="detail-title")
#     # Lưu văn bản của các thẻ 'detail-title' vào mảng
#     title_texts = [title.get_text(strip=True) for title in titles]
#     # Tách mã HTML thành các đoạn dựa trên các thẻ 'detail-title'
#     html_parts = []
#     for title in titles:
#         # Lấy nội dung của thẻ cha chứa 'detail-title' (nếu cần toàn bộ đoạn)
#         parent = title.find_parent()
#         # Xóa thẻ 'detail-title' trước khi lưu đoạn HTML vào html_parts
#         title.extract()
#         # Lưu đoạn HTML sau khi đã xóa thẻ 'detail-title'
#         if parent:
#             html_parts.append(str(parent))
#     PhucLoi = []
#     MoTaCongViec = None
#     yeuCauCongViec = None
#     ThongTinKhac = {}
#     for title, html in zip(title_texts, html_parts):
#         soup = BeautifulSoup(html, "html.parser")
#         if title.lower().strip() == 'Phúc lợi'.lower():
#             PhucLoi = get_phuc_loi(soup)
#         elif title.lower().strip() == 'Mô tả Công việc'.lower():
#             MoTaCongViec = get_info_job(soup)
#         elif title.lower().strip() == 'Yêu Cầu Công Việc'.lower():
#             yeuCauCongViec = get_info_job(soup)
#         elif title.lower().strip() == 'Thông tin khác'.lower():
#             ThongTinKhac = get_thong_tin_khac(soup)
#     dict = {
#         'PhucLoi': PhucLoi,
#         'MoTaCongViec': MoTaCongViec,
#         'yeuCauCongViec': yeuCauCongViec,
#         'ThongTinKhac': ThongTinKhac,
#     }
#     return dict

In [746]:
df = load_df()
# output = []
# hinh_thuc_set = {"Nhân viên chính thức"}
# cap_bac_set = {"Nhân viên"}
# kinh_nghiem_set = {"Chưa có kinh nghiệm"}
# phuc_loi_set = {"Chế độ bảo hiểm"}
for temp in range(0,10000):
    while True:
        try:
            ids = 901891053 + temp
            scraper = cloudscraper.create_scraper(
                            browser={
                                "browser": "chrome",
                                "platform": "windows",
                            },
                        )
            # specify the target URL
            url = f"https://careerviet.vn/vi/tim-viec-lam/chuyen-vien-xay-dung-bom-data.{format(ids, "x")}.html"
            response = scraper.get(url)
            print(response.status_code, ids)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                if get_heading1(soup):
                    info_job = get_info_job(soup)
                    data = {
                        'JobId': convert_value(extract_id_and_convert(get_url(soup))),
                        'UrlJob': convert_value(get_url(soup)),
                        'JobTitle': convert_value(get_heading1(soup)),
                        'IdCongTy': convert_value(get_info_company(soup)[0]), 
                        'UrlCongTy': convert_value(get_info_company(soup)[1]),
                        'TenCongTy': convert_value(get_info_company(soup)[2]),
                        'DiaDiem': get_dia_diem(soup),
                        'NganhNghe': get_info_post(soup).get('NganhNghe') or None,
                        'IsThoaThuan': IsThoaThuan(get_thong_tin_khac(soup).get('Luong')),
                        'mucluongfrom': processing_salary(get_thong_tin_khac(soup).get('Luong'))[0],
                        'mucluongto': processing_salary(get_thong_tin_khac(soup).get('Luong'))[1],
                        'currency': processing_salary(get_thong_tin_khac(soup).get('Luong'))[2],
                        'HinhThuc': get_info_post(soup).get('HinhThuc'),
                        'CapBac': get_info_post(soup).get('CapBac'),
                        'KinhNghiem': convert_value(get_info_post(soup).get('KinhNghiem')) or None,   
                        'NgayDangTin': get_info_post(soup).get('NgayCapNhat') or None,
                        'HanNopHoSo': get_info_post(soup).get('HetHanNop') or None,
                        'PhucLoi': get_phuc_loi(soup),
                        'MoTaCongViec': info_job.get('MoTaCongViec') or None,
                        'yeuCauCongViec': info_job.get('YeuCauCongViec') or None,
                        'DiaDiemLamViec': process_address_data(get_dia_chi(soup)[1], get_dia_chi(soup)[0], df) or process_address_data(get_dia_diem(soup), get_dia_diem(soup), df),
                        'ThongTinKhac': get_thong_tin_khac(soup),
                        'TuKhoa': get_key_word(soup),
                    }
                    output.append(data)
                    hinh_thuc_set.add(get_info_post(soup).get('HinhThuc'))
                    cap_bac_set.add(get_info_post(soup).get('CapBac'))
                    kinh_nghiem_set.add(convert_value(get_info_post(soup).get('KinhNghiem')) or None)
                    phuc_loi_set.update(get_phuc_loi(soup))
                    # print(json.dumps(data, ensure_ascii=False, indent=5, default=default_converter))
                    print(response.status_code, "Cập nhật thông tin thành công.", ids, len(output))
            if response.status_code == 429:
                    # print(f"ID {ids}: Too many requests, retrying...")
                time.sleep(random.randint(1, 5))  # Chờ một khoảng thời gian trước khi thử lại
            else:
                break  # Thoát khỏi vòng lặp nếu không phải 429
        except (requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
            print(f"ID {ids}: Lỗi kết nối {e}. Đang thử lại...")
            time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại

200 901891053
200 Cập nhật thông tin thành công. 901891053 393
200 901891054
200 Cập nhật thông tin thành công. 901891054 394
200 901891055
200 901891056
200 901891057
200 901891058
200 901891059
200 901891060
200 Cập nhật thông tin thành công. 901891060 395
200 901891061
200 901891062
200 901891063
200 Cập nhật thông tin thành công. 901891063 396
200 901891064
200 Cập nhật thông tin thành công. 901891064 397
200 901891065
200 901891066
200 901891067
200 Cập nhật thông tin thành công. 901891067 398
200 901891068
200 901891069
200 901891070
200 901891071
200 Cập nhật thông tin thành công. 901891071 399
200 901891072
200 Cập nhật thông tin thành công. 901891072 400
200 901891073
200 Cập nhật thông tin thành công. 901891073 401
200 901891074
200 Cập nhật thông tin thành công. 901891074 402
200 901891075
200 901891076
200 Cập nhật thông tin thành công. 901891076 403
200 901891077
200 901891078
200 Cập nhật thông tin thành công. 901891078 404
200 901891079
200 901891080
200 Cập nhật thông t

KeyboardInterrupt: 

In [749]:
# # Lưu vào tệp JSON
with open("sample_careerviet_30_11_2024_v3.json", "w", encoding="utf-8") as json_file:
    json.dump(output, json_file, ensure_ascii=False, indent=5, default=default_converter)

In [748]:
phuc_loi_set

{'CLB thể thao',
 'Chăm sóc sức khỏe',
 'Chế độ bảo hiểm',
 'Chế độ thưởng',
 'Công tác phí',
 'Du Lịch',
 'Du lịch nước ngoài',
 'Laptop',
 'Nghỉ phép năm',
 'Phụ cấp',
 'Phụ cấp thâm niên',
 'Tăng lương',
 'Xe đưa đón',
 'Đào tạo',
 'Đồng phục'}

In [747]:
hinh_thuc_set

{'',
 'Bán thời gian',
 'Bán thời gian, Thời vụ/ Nghề tự do',
 'Bán thời gian, Thực tập',
 'Nhân viên chính thức',
 'Nhân viên chính thức, Bán thời gian',
 'Nhân viên chính thức, Bán thời gian, Thời vụ/ Nghề tự do',
 'Nhân viên chính thức, Thời vụ/ Nghề tự do',
 'Nhân viên chính thức, Thực tập',
 None,
 'Thời vụ/ Nghề tự do',
 'Thực tập'}

In [744]:
kinh_nghiem_set

{'1 - 10 Năm',
 '1 - 2 Năm',
 '1 - 3 Năm',
 '1 - 4 Năm',
 '1 - 5 Năm',
 '10 - 15 Năm',
 '10 - 20 Năm',
 '2 - 10 Năm',
 '2 - 3 Năm',
 '2 - 4 Năm',
 '2 - 5 Năm',
 '3 - 4 Năm',
 '3 - 5 Năm',
 '3 - 6 Năm',
 '3 - 7 Năm',
 '4 - 6 Năm',
 '4 - 7 Năm',
 '5 - 15 Năm',
 '5 - 6 Năm',
 '5 - 7 Năm',
 '5 - 8 Năm',
 '6 - 10 Năm',
 '8 - 10 Năm',
 'Chưa có kinh nghiệm',
 'Lên đến 1 Năm',
 'Lên đến 15 Năm',
 'Lên đến 2 Năm',
 None,
 'Trên 1 Năm',
 'Trên 10 Năm',
 'Trên 2 Năm',
 'Trên 3 Năm',
 'Trên 5 Năm',
 'Trên 6 Năm',
 'Trên 7 Năm',
 'Trên 8 Năm'}

In [745]:
cap_bac_set

{'Giám đốc',
 'Mới tốt nghiệp',
 'Nhân viên',
 None,
 'Phó Giám đốc',
 'Quản lý',
 'Sinh viên/ Thực tập sinh',
 'Trưởng nhóm / Giám sát'}

In [ ]:
# scraper = cloudscraper.create_scraper(
#                             browser={
#                                 "browser": "chrome",
#                                 "platform": "windows",
#                             },
#                         )
# # specify the target URL
# url = f"https://careerviet.vn/vi/tim-viec-lam/kcn-long-hau-long-an-truong-phong-bao-tri-20tr-30tr-25036.35C23307.html"
# response = scraper.get(url)
# print(response.status_code, ids)
# if response.status_code == 200:
#     soup = BeautifulSoup(response.text, 'html.parser')
#     if get_heading1(soup):
#         info_job = get_info_job(soup)
#         data = {
#             'JobId': convert_value(extract_id_and_convert(get_url(soup))),
#             'UrlJob': convert_value(get_url(soup)),
#             'JobTitle': convert_value(get_heading1(soup)),
#             'IdCongTy': convert_value(get_info_company(soup)[0]), 
#             'UrlCongTy': convert_value(get_info_company(soup)[1]),
#             'TenCongTy': convert_value(get_info_company(soup)[2]),
#             'DiaDiem': get_dia_diem(soup),
#             'NganhNghe': get_info_post(soup).get('NganhNghe') or None,
#             'IsThoaThuan': IsThoaThuan(get_thong_tin_khac(soup).get('Luong')),
#             'mucluongfrom': processing_salary(get_thong_tin_khac(soup).get('Luong'))[0],
#             'mucluongto': processing_salary(get_thong_tin_khac(soup).get('Luong'))[1],
#             'currency': processing_salary(get_thong_tin_khac(soup).get('Luong'))[2],
#             'HinhThuc': get_info_post(soup).get('HinhThuc'),
#             'CapBac': get_info_post(soup).get('CapBac'),
#             'KinhNghiem': convert_value(get_info_post(soup).get('KinhNghiem')) or None,   
#             'NgayDangTin': get_info_post(soup).get('NgayCapNhat') or None,
#             'HanNopHoSo': get_info_post(soup).get('HetHanNop') or None,
#             'PhucLoi': get_phuc_loi(soup),
#             'MoTaCongViec': info_job.get('MoTaCongViec') or None,
#             'yeuCauCongViec': info_job.get('YeuCauCongViec') or None,
#             'DiaDiemLamViec': process_address_data(get_dia_chi(soup)[1], get_dia_chi(soup)[0], df) or process_address_data(get_dia_diem(soup), get_dia_diem(soup), df),
#             'ThongTinKhac': get_thong_tin_khac(soup),
#             'TuKhoa': get_key_word(soup),
#         }
#         output.append(data)
#         print(json.dumps(data, ensure_ascii=False, indent=5, default=default_converter))